- Helper to AutomotiveInteraction.jl
- Perform non-linear least squares fit using Levenberg-Marquardt
- Adapted from Jeremy Morton's code in LSTM-acc-predict
- The IDM features and true acceleration value is provided by the Julia code in the form of .txt file
- Should think about integrating python part i.e. this file directly using PyCall but leave for later

- How this works
    - Reads in idm feature values from `../lmfit/scenario_name/vehid_idmfeats.txt`
    and true acceleration from `../lmfit/scenario_name/vehid_acctrace.txt`
    - Calls lmfit.minimize to run the Leveberg-Marquardt algorithm for fitting
    - Writes resulting parameters in the order `v_des`,`T`,`dmin` in `../lmfit/scenario_name/vehid_lmfit_params.txt`

In [11]:
# imports
from lmfit import minimize, Parameters, Parameter, report_fit
import math
import numpy as np
import os

In [7]:
# Function to minimize
def IDM(params, i, true_acc):
    dmn = params['dmn'].value
    T = params['T'].value
    bcmf = params['bcmf'].value
    smx = params['smx'].value
    amx = params['amx'].value
    
    d = x[i, 0]
    r = x[i, 1]
    s = x[i, 2]
    
    d_des = dmn + T*s - s*r/2/math.sqrt(amx*bcmf)
    pred = amx*(1 - (s/smx)**4 - (d_des/d)**2)
    
    return pred - true_acc

In [8]:
# Initialize parameter values
params = Parameters()
params.add('dmn', value = 6., min = 0.1, max = 5.25) 
params.add('T', value = 1., min = 0.1, max = 5.0)
params.add('bcmf', value = 1.0, min = 0.5, max = 5.0) 
params.add('smx', value = 15.0, min = 1.0, max = 30.0)
params.add('amx', value = 1.0, min = 0.75, max = 3.0)

In [9]:
# Upper scenarios: Specify the list of scenarios
scenarios_upper = [
        [6,8,13,19,28,29],
        [34,37,40,42,43,49,50],
        [59,60,62,66,67,71,73],
        [66,67,71,73,76,82,88,92,95],
        [187,191,193,194],

        [211,213,215,217,221],
        [248,249,250,256],
        [272,275,276,277,279],
        [294,295,300,307,308,311],
        [324,327,329,333,335,339,341,346,354,357]
]

In [23]:
# Loop over the scenarios. Save fit params in eg: `lmfit/upper_1/veh_id_lmfit_params.txt
for j in range(0,len(scenarios_upper)):
        id_list = scenarios_upper[j]
        i = j+1 # because used to read files so can't start from 0
        scenario_name="upper_{}".format(i)
        print("name = {}".format(scenario_name))
        print("id_list = {}".format(id_list))
        
        for vehid in id_list:
            print("vehicle id = {}\n".format(vehid))
            idm_feat_filename = "../lmfit/{}/{}_idmfeats.txt".format(scenario_name,vehid)
            if os.stat(idm_feat_filename).st_size != 0:
                print("Features file is not empty. Filename = {}".format(idm_feat_filename))
                x = np.loadtxt(idm_feat_filename)
                acc = np.loadtxt("../lmfit/{}/{}_trueacc.txt".format(scenario_name,vehid))
                # Perform the fit
                result = minimize(IDM,params,args=(range(len(acc)),acc))
                paramvals = np.array([result.params['smx'].value,
                                      result.params['T'].value,result.params['dmn'].value])
                np.savetxt("../lmfit/{}/{}_lmfit_params.txt".format(scenario_name,vehid),paramvals)

name = upper_1
id_list = [6, 8, 13, 19, 28, 29]
vehicle id = 6

Features file is not empty. Filename = ../lmfit/upper_1/6_idmfeats.txt
vehicle id = 8

Features file is not empty. Filename = ../lmfit/upper_1/8_idmfeats.txt
vehicle id = 13

vehicle id = 19

Features file is not empty. Filename = ../lmfit/upper_1/19_idmfeats.txt
vehicle id = 28

Features file is not empty. Filename = ../lmfit/upper_1/28_idmfeats.txt
vehicle id = 29

Features file is not empty. Filename = ../lmfit/upper_1/29_idmfeats.txt
name = upper_2
id_list = [34, 37, 40, 42, 43, 49, 50]
vehicle id = 34

Features file is not empty. Filename = ../lmfit/upper_2/34_idmfeats.txt
vehicle id = 37

Features file is not empty. Filename = ../lmfit/upper_2/37_idmfeats.txt
vehicle id = 40

vehicle id = 42

Features file is not empty. Filename = ../lmfit/upper_2/42_idmfeats.txt
vehicle id = 43

Features file is not empty. Filename = ../lmfit/upper_2/43_idmfeats.txt
vehicle id = 49

Features file is not empty. Filename = ../lmfit/up